# NoSQL DBs - MongoDB

## 00 - Introduction

As we discussed, NoSQL DBs can fall under many sub-categories depending on the data model, e.g. key-value store, graph, wide-column, ...

All offer a inherent flexbility to representing data with respect to DBs relying on the Relational model.

Probably the most flexible approach to store and retrieve complex data is the one provided Document-oriented DBs, and among those, MongoDB is defintely the most well-known and used as of today.

https://db-engines.com/en/ranking_trend/document+store

Furthermore, MongoDB finds its place among the top 5 used DBMSs overall, topping the list of non-relational DBs.

https://www.statista.com/statistics/809750/worldwide-popularity-ranking-database-management-systems/

https://insights.stackoverflow.com/survey/2021#most-popular-technologies-database

MongoDB was originally implemented in C++, and offers a number of interfaces to work with.

We will focus on `pymongo`, the de-facto standard module to interact with MongoDB.

## 01 - Connecting to the DB and Creating collections

As for the MySQL example, a MongoDB server is already running in a docker container, mimicking an existing server to which we will connect to insert and query our databases.

The following users are created by default during the docker container spawn phase:

A `root` user is created by default with:
- user: `root`
- password: `root_pwd`

A non-`root` user is also created by default with:
- user: `my_user`
- password: `user_pwd`

#### Interactive shell

MongoDB offers a shell to allow interacting with the server directly: `mongosh`
    
We will use it very unfrequently, but it is useful to know how to access it.

We can log into the server by attaching to the running `CONTAINER ID` of the container named `mongo_db`:
    
```bash 
$ docker ps 
...
...
...
$ docker exec -it <YOUR CONTAINER ID> bash
```

And from inside the container

```bash
$ mongosh -u <EITHER ROOT OR USER> -p
``` 

Once connected to the server we can check the list of databases with:
    
```python
show databases
```

_NB_: the `mongosh` shell is quite permissive, so we don't need to end all statements with semicolons `;`

#### `pymongo`

The `pymongo` module is already installed in your docker Jupyter-notebook container.

_NB_: remember that we use the containers to replicate a server-client setup. The server is hosted in a different container from the Jupyter notebook, therefore we need to connect to the proper server IP address (in our case, simply `db`).

In [ ]:
import pymongo
from pymongo import MongoClient

from pprint import pprint

import matplotlib.pyplot as plt

Create a connection to the DB.

We create a client connecting to the MongoDB server, which can be, depending on the deployment of the DB:
- the single node hosting the DB
- the primary of a replica set
- the router of the sharded cluster

On the node running MongoDB there should be an active `mongod` server to which we will connect.

This has been done automagically at the startup of the mongo docker container, but one will have to do it by hand in a different configuration (e.g. when deploying MongoDB in a non docker-based environment, or if you wanted to have a MongoDB server running on your laptop).

Luckily, MongoDB documentation is quite clear and detailed:
- [standalone](https://docs.cloudmanager.mongodb.com/tutorial/deploy-standalone/)
- [replica set](https://docs.cloudmanager.mongodb.com/tutorial/deploy-replica-set/)
- [sharded cluster](https://docs.cloudmanager.mongodb.com/tutorial/deploy-sharded-cluster/)

In [ ]:
# declare a client connection to the MongoDB server 
dbc = MongoClient(username="root", 
                  password="root_pwd",
                  host="db", # in this case 'db' is the equivalent of the IP address of the server 
                             # or 'localhost' if running locally on the MongoDB server node
                 )

The client allows to inspect the content of the server, depending on the permissions associated to the authenticated user.

When connecting via as the `root` user, with administration rights over the MongoDB server(s), we can see that a number of DBs are already present by default:

These are the way MongoDB stores all information concerning users, status of the nodes, etc...
- `admin` plays a role in authentication and authorization
- `local` stores data specific to a single server, and info used in the replication process in replica sets deployment
- `config` store information about each shard, in the sharded deployment

In [ ]:
print(dbc.list_database_names())

print('---')

for _ in dbc.list_databases():
    print(_)

If connecting via a non-root user, no db should be visible by default, due to the authorization restraints following the authentication:

In [ ]:
dbu = MongoClient(username="my_user", 
                  password="my_pwd",
                  host='db', 
                 )

print(dbu.list_database_names())

print('---')

for _ in dbu.list_databases():
    print(_)

To use or to create a DB via the pymongo API, we can simply create a new "database object" by issuing the following:

```python
<DB_OBJECT> = <CLIENT>['<DB_NAME>']
```

or

```python
<DB_OBJECT> = <CLIENT>.<DB_NAME>
```

If the DB already exists, we will have pointed the newly created "database object" to it.
We can thus start inspecting its content and play with its collections.

If the DB did not exist beforehand, MongoDB will ***plan*** to create it for us.
The new DB will not be automatically created, though... 
The DB will "materialize" only once we will have inserted some content (at least 1 record) to it.


In [ ]:
db = dbc['my_db']

Now we should have successfully created a `db` database object.

First, let's check once more the list of DBs hosted in mongo.

In [ ]:
print(dbc.list_database_names())

print('---')

for _ in dbc.list_databases():
    print(_)

We can now start using the db object directly.

The pymongo API will allow to inspect, create, and populate the collections easily from the `db` object.

To list the collections contained in the DB:
    
```python
db.list_collection_names()
```

or 

```python
db.list_collections()
```

In [ ]:
print(db.list_collection_names())

print('---')

for _ in db.list_collections():
    pprint(_)

Collections are created and accessed in a similar way to the creation of a db.

```python
<COLLECTION_OBJECT> = <DB_OBJECT>['<COLLECTION_NAME>']
```

or

```python
<COLLECTION_OBJECT> = <DB_OBJECT>.<COLLECTION_NAME>
```

As for the DB creation, the creation of a new collection is a ***lazy operation***, i.e. it's not done explicitely until you fill the collection with your first document.


In [ ]:
col = db['my_collection']

In [ ]:
print(db.list_collection_names())

print('---')

for _ in db.list_collections():
    pprint(_)

## 02 - Inserting Documents

To verify that a DB and a collection are in fact created with the insertion of the first document we can proceed to issue a DB CRUD transaction by inserting a single document in the DB.

We can do it via either by explicitely using the `collection_object`, or by referring to the collection via `database_object.collection_name` 

```python
<DB_OBJECT>.<COLLECTION_NAME>.insert_one(<DOCUMENT>)
```

or 

```python
<COLLECTION_OBJECT>.insert_one(<DOCUMENT>)
```

In MongoDB all documents are stored as BSON, binary JSON objects.

We can insert any JSON record in MongoDB, withouth any need to pre-define the schema of the collection (~table for a RDB) or the entire DB.

Every document is identified by the document `_id`.

Please note that:
- the `_id` field can be declared explicitely
- otherwise, it is defined automatically by MongoDB upon the insertion of the record

Using the `pymongo` API, we can use a `python` dictionaries as documents.

Create 1 document entry and insert it to the newly crated collection.
- let `pymongo` assign the `_id` field automatically
- perform the insertion using the `collection_object` previously created

In [ ]:
dict_ = {"field_1": "A string", 
         "field_2": 12.5,
         "field_3": [2,'1','asd'],
         "field_4": {
             "field_4-1": "foo",
             "field_4-2": "bar"
         },
        }

col.insert_one(dict_)

Create a second document.
- assign the `_id` field explicitely with an arbitrary string value
- perform the insertion calling the collection by name from the `db_object`

The value of the `_id` field can be anything you like, e.g. a string.

In [ ]:
dict_ = 

# you can also store the result of the insertion in a variable
_ = db.my_collection.insert_one(dict_)

At this point both the DB and the collection previosly instantiated are actually created, and are visible when listing the server content

In [ ]:
# print the list of dbs and the list of collections

We can verify that the collection is now hosting the documents we have created by issuing a `find()` on the collection.

The `find()` will produce the results of retrieving all content from the collection.

The output will be a `cursor` object, that we can use to iterate on the results.

In [ ]:
for _ in db.my_collection.find():
    pprint(_)

The default `_id` is a `ObjectId`, a unique identifier of BSON format with a fixed-size of 12-bytes.

The `_id` field is a complex object created automatically by combining a number of information, including:
- the timestamp of the insertion
- the MongoDB node identifier
- an identifier of the current process 
- a counter

```bash
621774855e223c0dc836f720
TS------ID----PID-Cnt---
```

We can however redefine the `_id` depending on our application to represent any kind of data format:
- binary data
- integer
- string
- timestamp
- ...

The document `_id` can be used to retrieve documents directly, similarly to what could be done in a key-value store, or can be simply used as an internal bookkeeping info for MongoDB.

Let's stress this once more:
- MongoDB allows 2 identical documents, with different `_id` fields
- but do not allow 2 different documents, with identical `_id` fields

Try to insert 2 new documents:
1. with a different `_id` field from the previous ones
2. with the same `_id` field from a previous document

Finally, print the content of the collection

In [ ]:
# different _id field wrt previous document 
# same field_1 and field_2 as the previous document

dict_1 = {"_id":    # fill here
         "field_1": # fill here
         "field_2": # fill here
        }

try:
    db.my_collection.insert_one(dict_1)
    print("Successful document insertion")
except:
    print("An exception occurred")


In [ ]:
# same _id field wrt previous document
# completely different document content

dict_2 = {"_id":          # fill here
         "new_field":     # fill here
         "another_field": # fill here
        }

try:
    db.my_collection.insert_one(dict_2)
    print("Successful document insertion")
except:
    print("An exception occurred")

List all documents inserted to check the content of the DB at this stage

In [ ]:
# find all documents in the collection

We can insert multiple documents at once via the `insert_many` method of the `pymongo` API.

The documents have to be passed as a `python` list:
    
```python
<DB_OBJECT>.<COLLECTION_NAME>.insert_many([<DOCUMENT1>,
                                           <DOCUMENT2>,
                                           <DOCUMENT3>])
```

or 

```python
<COLLECTION_OBJECT>.insert_many([<DOCUMENT1>,
                                 <DOCUMENT2>,
                                 <DOCUMENT3>])
```

Create a small set of documents and insert them in a single batch.

Store the result of the transaction in a new python variable.

In [ ]:
from random import random

docs = []
for _ in range(3):
    docs.append({'_id': _,
                 'x': random(),
                 'y': random()
                })

# store the result of the transaction as a new object
res_ = db.my_collection.insert_many(docs)

pprint(res_)

The result variable contains information:
1. the newly inserted `_id`s
2. the aknowledgement response of the MongoDB server to the insertion

In [ ]:
print('inserted ids:', res_.inserted_ids)
print('transfer ack:', res_.acknowledged)

## 03 - Basics on finding objects 

To query the DB and check the content of the inserted documents, the `find` method acts as the `SELECT` instruction in SQL.

```python
<DB_OBJECT>.<COLLECTION_NAME>.find()
```

or 

```python
<COLLECTION_OBJECT>.find()
```

A quite useful alternative method is `find_one`, which acts in the very same way as `find` but selects and shows only *one* document of the collection 

- retrieve all documents previously inserted in the collection with `find`
- use `find_one` to inspect one of the documents

In [ ]:
# find

In [ ]:
# find_one

The `find` methods can be modified to return only a subsets of fields from the documents, to perform the equivalent to the SQL statement

```mysql
SELECT Attribute1, Attribute2 FROM Table;
```

In pymongo this can be achieved by enabling (1) or disabling (0) individual fields:
    
```python
<DB_OBJECT>.<COLLECTION_NAME>.find({}, 
                                   {"FIELD1":1, "FIELD2":1})
```

This operation is referred to as document *projection* over the specific fields.

**NB 1**: a peculiar quirk of MongoDB makes only possible to specify which fields to drop or to retain with either only 0 or only 1 values, and not to mix 0s and 1s in the same `find`

**NB 2**: the latter does not apply for the `_id` field, which can however be always silenced 


In [ ]:
# we can do the following
try:
    _ = db.my_collection.find({},
                              {'_id':0,
                               'x'  :1,
                               'y'  :1})
    pprint([x for x in _]) 
except:
    print("An exception occurred")    

In [ ]:
# but we cannot do the following
try:
    _ = db.my_collection.find({},
                              {'_id':0,
                               'x'  :1,
                               'y'  :0})
    pprint([x for x in _]) 
    print("Successful query")
except:
    print("An exception occurred")    

Let's import a simple DB from a JSON file to start working on queries.

DBs and collections can be imported using the `mongoimport` tool directly connecting to the mongo server.
This is the preferred way, and the best alternative for large databases dumped into files.

```bash
mongoimport --username               <USER NAME> 
            --password               <USER PASSWORD>
            --db                     <DB NAME>
            --collection             <COLLECTION NAME> 
            --authenticationDatabase admin
            --file                   <.json FILE PATH>
```

As an pythonic alternative for smaller datasets, we can proceed in pymongo by:
1. creating a new DB 
2. creating a new collection
3. creating a list of documents by reading the .json file 
4. performing the insertion of all documents in the newly created collection

**NB**: when importing a json file produced by the dump of a MongoDB database, some object fields may not be interpreted correclty by the `python` json module. This is because MongoDB renames and modifies some fields (e.g. `ObjectId` into `$oid`) during the dump to json format.

We are going to use a DB from businesses rated on YELP, stored in the file: `/opt/workspace/dbs/yelp.json`

1. create a new DB named 'yelp'
2. create a new collection named 'businesses'
3. create an empty list of documents
4. loop over the .json file and append every line to the document list 
  * use the `json` module to append the `json.loads(<THE LINE>)`
5. perform an `insert_many` transaction to populate the database

In [ ]:
# if necessary, drop the existing yelp database

In [ ]:
# create a new db

In [ ]:
# create a new collection

In [ ]:
# create a list of documents 

In [ ]:
# read documents from the json file and fill the document list

In [ ]:
# insert the documents in the db

In [ ]:
# inspect the inserted data (one document)

## 03 - Basic queries with filters

As we anticipated earlier, the `find` statement API is composed of 2 parts:
1. the query operator(s)
2. the projection

```python
<DB_OBJECT>.<COLLECTION_NAME>.find(
                                   {<QUERY OPERATORS>}, 
                                   {<DATA PROJECTION>}
                                  )
```

All query operators are passed to pymongo as a dictionaries (jsons), or a list of dictionaries.

To retrieve all documents with a specific value at a given field the json query operator is simply formatted as:

```python
{'<THE FIELD>' : 'THE VALUE'}
```

* Locate all businesses in the `state` of Florida (FL)
    * Only show the first 2 retrieved documents (by limiting the loop over the cursor)

In [ ]:
# query 
q_ = {'state' : 'FL'}

# projection
p_ = {}

try:
    _ = db_yelp.businesses.find(q_, p_)
    for i in range(2):
        pprint(_[i])
except:
    print('Failed query')

MongoDB provides a set of operators to perform more extensive queries (https://docs.mongodb.com/manual/reference/operator/query/) such as:

```python
$eq    # equal to
$ne    # not equal to
$gt    # greater than
$gte   # greater than or equal to
$lt    # less than
$lte   # less than or equal to
$in    # in the list
$nin   # not in the list
...
```

The syntax of the query is still a json-like format, with nested conditions:

```python
{'<THE FIELD>' : { 'THE CONDITION' : 'THE VALUE'} }
```

* Retrieve the `name`, `city` and `review_count` of all businesses with more than 100 `review_count`

More articulated queries can be performed by combining multiple conditions in a single query statement (i.e. one single json-like query).
By default, this will result in the boolean `and` of all query statements.
    
```python
{'<THE FIELD 1>' : { 'THE CONDITION 1' : 'THE VALUE 1'},
 '<THE FIELD 2>' : { 'THE CONDITION 2' : 'THE VALUE 2'} }
```    

* Retrieve the `name`, `city`, `review_count` and `stars` of all businesses with more than 100 `review_count` and with 4 or more `stars`

To chain further query operators in a single statement we can use boolean operators.

```python
$and
$or
$not
$nor
```

The syntax of the query highlights the boolean operator, and applies to a list of sub-conditions, in the order:
1. the boolean operator (e.g. `$nor :`)
2. the list of all conditions affected `[ {<CONDITION 1>}, {<CONDITION 2>} ]`


```python
{'<BOOLEAN OPERATOR>' : [ { 'THE CONDITION 1' : 'THE VALUE 1'},
                          { 'THE CONDITION 2' : 'THE VALUE 2'} ] }
```    

* Perform a similar query as before by selecting the `name`, `city`, `review_count` and `stars` of all businesses with more than 100 `review_count` ***or*** with 4 or more `stars`


* Then, select `name`,`city`,`state` and `stars` of all businesses with 4 or more `stars` that can be found in the `state` of Colorado (CO) or Washington (WA)

There are 2 main alternatives in this case:
1. perform the **or** between the two `state` conditions, followed by the **and** of the `stars` condition
2. use the `$in` statement to search for elements in a list

So far we have limited the output of the query by stopping displaying the results when iterating on the database cursor.

A better way to limit the results is to apply the `limit` operator directly in the query statement, thus returing only a limited amount of documents.

This can be further combined with the sorting (`sort` operator).
Differently from plain MongoDB, in `pymongo`:
1. the sorting conditions are reported in a list of tuples `[(...), (...)]`
2. the sorting direction can be stated explicitly via `pymongo.ASCENDING/DESCENDING` instead of using 1 and -1

```python
<DB_OBJECT>.<COLLECTION_NAME>.find( {<QUERY OPERATORS>}, {<DATA PROJECTION>} ) \
                             .sort( [(<FIELD 1>, pymongo.ASCENDING / 1),
                                     (<FIELD 2>, pymongo.DESCENDING / -1)] ) \
                             .limit(N)
```

* Select the first 5 businesses in the `state` of Pennsylvania (PA) with the highest `stars` rating
    * Further sort them by `city`
    * Show `name`,`city`,`state` and `stars` of all resulting documents

## 04 - Queries on Nested Fields

MongoDB documents are easily nested, hosting a number of inner structures which allow us to store and organize the data more clearly.

For instance, most documents (but not necessarily all of them) in this collection have a `attributes` data structure, whose elements are further structured, e.g.:

```python
 'attributes': {'BikeParking': 'True',
                'BusinessAcceptsCreditCards': 'True',
                'BusinessParking': {'garage': 'False',
                                    'lot': 'False',
                                    'street': 'False',
                                    'valet': 'False',
                                    'validated': 'False'},
                'RestaurantsPriceRange2': '2'},
```

We can perform queries of the sub-fields of nested objects simply by accessing data with the "dot" notation `field.sub_field`

* Retrieve the five best rated businesses from Florida (FL) with at least 10 reiews with the `attributes` `GoodForKids` being equal to `'True'` 
    * Choose freely the limit and sorting of the data

With the query operators discussed so far, use the `businesses` collection of the YELP database to extract the following information:
    
1. Retrieve the name and state of the first 10 businesses with more than 2000 reviews
    * Sort the results by stars in descending order


2. Locate all businesses in Nevada (NV) or Luisiana (LA) with more than 2.5 and less than 5.0 stars ratings.
    * Sort by the number or ratings in descending order
    * Limit the query to the first 10 occurrencies
    

3. List the addresses of the first 5 businesses currently open (`is_open`) in Reno (NV) with BikeParking as an Attribute
    * Sort by stars
    

4. Find the top 5 best rated businesses having Friday `11:0-18:0` within their opening `hours`


5. Identify the most voted business in Santa Barbara (CA) offering the `BusinessParking.valet` service


6. Select the 5 best rated business in any of the follwing states (New Jersey, Indiana, Tennessee) with more than 50 ratings where `GoodForMeal.lunch` and `DogsAllowed` are both true, and where the `Ambience` is not `touristy`

#### 1 - Retrieve the name and state of the first 10 businesses with more than 2000 reviews, sorted by stars in descending order

#### 2 - Locate all businesses in Nevada (NV) or Luisiana (LA) with more than 2.5 and less than 5.0 stars ratings, sorted by the number or ratings in descending order; limit the query to the first 10 occurrencies

#### 3 - List the addresses of the first 5 businesses currently open (`is_open`) in Reno (NV) with BikeParking as an Attribute, sorted by stars

#### 4 - Find the top 5 best rated businesses having Friday `11:0-18:0` within their opening `hours`

It is possible to use Pandas to visualize the results of the queries.

From a query we return a cursor object containing a "pointer" to the list of json-like results (dictionaries).

In Pandas, we can create a DataFrame out of it by doing the following:
    
```python
import pandas as pd
df = pd.DataFrame(list(<QUERY RESULT>))
```

In [ ]:
import pandas as pd

try:
    _ = db_yelp.businesses.find(q_, p_).sort(s_).limit(l_)
    df = pd.DataFrame(list(_))
except:
    print('Failed query')
    
df

#### 5 - Identify the most voted business in Santa Barbara (CA) offering the `BusinessParking.valet` service

In [43]:
# import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline 

# try:
#     _ = db_yelp.businesses.find(q_, p_).sort(s_).limit(l_)
#     df = pd.DataFrame(list(_))
# except:
#     print('Failed query')
    
# plt.scatter('review_count', 
#             'stars', 
#             data=df)

#### 6 - Select the 5 best rated business in any of the follwing states (New Jersey, Indiana, Tennessee) with more than 50 ratings where `GoodForMeal.lunch` and `DogsAllowed` are both true, and where the `Ambience` is not `touristy`

## 05 - Queries on Arrays and List Elements

Queries of fields containing lists can be performed with the same syntax of any other field.

In a list, however, the order of the elements is extremely important:

```python
list_1 = ['a','b','c'] 
list_1[0] = 'a'
```

```python
list_2 = ['c','b','a'] 
list_2[0] = 'c'
```

This is reflected in the search statements, as the following two queries will return a completely different result:

```python
<DB_OBJECT>.<COLLECTION_NAME>.find( { <LIST FIELD> : [<ELEMENT A>, <ELEMENT B>] } )
```

```python
<DB_OBJECT>.<COLLECTION_NAME>.find( { <LIST FIELD> : [<ELEMENT B>, <ELEMENT A>] } )
```

When searching for all documents containing an element in any possible position within an array, the following, less stringent, notation is allowed:

```python
<DB_OBJECT>.<COLLECTION_NAME>.find( { <LIST FIELD> : <ANY ELEMENT> } )
```

If the specific ordering of the list is instead known, the query can be performed by explicitly addressing the i-th element of the list using the dot notation:

```python
<DB_OBJECT>.<COLLECTION_NAME>.find( { <LIST FIELD.i> : <i-th ELEMENT> } )
```

Test the query on lists by searching the businesses where the list `categories` have:

(For all queries, sort by stars in descending order and show only the name of the 5 best rated businesses, together with their categories)

1. The terms `Doctors` and `Health & Medical` in this specific order


2. The terms `Health & Medical` and `Doctors` in this specific order


3. The term `Doctors` anywhere in the list


4. The term `Doctors` as the 3rd element of the list




In [ ]:
# 1.

In [ ]:
# 2.

In [ ]:
# 3.

In [ ]:
# 4.

## 06 - More on Basic Queries

Let's combine all basic filters in more articulated queries using the `mflix` database.

Preliminary operations:
* Import the `/opt/workspace/dbs/movies.json` file in the `mflix` databse and `movies` collection 
    * The file contains `$oid` structure instead of `ObjectId` as `_id` field, so the best alternative is to import it via `mongoimport` (from the `mongo` container, with the command presented here [*])
* Create the database cursor object and/or the associated collection object 
* Inspect a few documents to understand the intrinsic "schema" of the records

[*]
```bash
mongoimport --username               <USER NAME> 
            --password               <USER PASSWORD>
            --db                     <DB NAME>
            --collection             <COLLECTION NAME> 
            --authenticationDatabase admin
            --file                   <.json FILE PATH>
```

In [ ]:
# create the mflix database

In [ ]:
# create the movies collection 

In [ ]:
# import the DB from mongosh [see instructions]

In [ ]:
# inspect one document

1. Select the 5 highest imdb-rated movies produced after 1990 with at least one award nomination
    * Let's stop here and disuss all together the results...


2. Select any film with genre `Short` with a rotten tomatoes rating greater than 8.5. Sort them by descending year and Rotten Tomatoes meter
    * List the first 10


3. Identify the films with `Christopher Nolan` as the one and only director (no co-directors). Which one of them was the oldest released to win at least 2 awards?


4. Find all movies by `Peter Jackson` with genre `Fantasy` or `Adventure` with a Rotten Tomatoes meter for both Critic and Viewers greater than 80. 
    * Do you think you will find the `The Hobbit` movies in this list?

#### 1 - Select the 5 highest imdb-rated movies produced after 1990 with at least one award nomination

We have a duplicate entry!
`The Shawshank Redemption` is present twice in this list. 

We might want to can get rid of duplicates with the MongoDB equivalent to the SQL `DISTINCT` statement.

In pymongo a `distinct` method can be used as an ***alternative*** to `find`
    
```python
<DB_OBJECT>.<COLLECTION_NAME>.distinct(
                                   {<FIELD TO ACT WITH DISTINCT>}, 
                                   {<FILTER OPERATION>}
                                  )
```

**However**, quite unfortunately we cannot easily combine a `distinct` statement with a `sort` or a `limit`. For that we need to use *aggregations*, which will be discussed later on...

In [ ]:
# distinct field 
d_ = 'title'

try:
    _ = db_mflix.movies.distinct(d_,q_)
    for x in _:
        pprint(x)
except:
    print('Failed query')

#### 2 - Select any film with genre `Short` with a rotten tomatoes rating greater than 8.5. Sort them by descending year and Rotten Tomatoes meter; and list the first 10.

#### 3 - Identify the films with `Christopher Nolan` as the one and only director (no co-directors). Which one of them was the oldest released to win at least 2 awards?

#### 4 - Find all movies by `Peter Jackson` with genre `Fantasy` or `Adventure` with a Rotten Tomatoes meter for both Critic and Viewers greater than 80.  

## 07 - Text-based Queries

MongoDB allows to perform string-based queries in text fields in 2 ways:
- with Regular Expressions (regex)
- with full-text searches

A regular expression is a sequence of characters that allows to identify and select a **pattern** in a string, instead of a specific substring.
Most editors (including Jupyter) allow to use regex to find and replace text based on patterns, and several interesing sites offer a simple interface to learn and experiment with regexp, e.g. https://regex101.com/

All following charachters are considered "meta-characters" in regex

`. ^ $ * + ? { } [ ] \ | ( )`

In a regular expression, all these symbols will be **not** interpreted as their characters, but are used to build up the patterns and search for matches in the text.

The main notable examples are:

`[ ]` square brakets meaning exactly 1 character

| regexp | interpretation |
| --- | --- |
| \[a\] | the letter `a` |
| \[ax\] | the letter `a` or `x` |
| \[a-x\] | any letter from `a` to `x` |
| \[a-zA-F\] | any letter from `a` to `z` and from `A` to `F` |
| \[0-9\] | any number from 0 to 9 |

`^` represents a `not`

| regexp | interpretation |
| --- | --- |
| \[^b02\] | all charachters but `b`,`0`and `2` |

`.` represents any character at all

| regexp | interpretation |
| --- | --- |
| . | all charachters |

`?` makes the preceding character in the regular expression optional

| regexp | interpretation |
| --- | --- |
| colou?r | `color` or `colour` |

`*` match the preceding character (or combination) zero or more times

| regexp | interpretation |
| --- | --- |
| go*gle | `ggle`,`gogle`,`google`,`gooogle`,... |

`\t` matches a tab

`\n` matches a new-line

`^` (outside of `[]`) matches the start of a line

`$` matches the end of a line

Regex can be used in plain Python with the `re` module.



In MongoDB we can search for patterns across entire bodies of text thanks to the `$regex` find operator.
All documents containing the pattern in the specified will be returned.


```python
{'<THE FIELD>' : { '$regex' : 'THE PATTERN'} }
```

or

```python
{'<THE FIELD>' : { '$regex' : /THE PATTERN/} }
```

Let's use regex to find all occurrencies of simple strings in a body of text, although this is a slight mis-use of the power of regex...

1. Search for all `Indiana Jones` movies using regex


2. Plot the Metacritic score of all `Harry Potter` movies. Identify the movies using regex


3. Inspect the dataset to search for all mentions of movies that won one or more Oscar award(s) but are judged poorly by the viewers (Rotten Tomatoes viewer meter < 50). 
    * Select the five movies with the highest total number of awards won.
    * (you can find the award mentioned under the `text` subfield of the `awards` field)

#### 1 - Search for all `Indiana Jones` movies using regex

#### 2 - Plot the Metacritic score of all `Harry Potter` movies. Identify the movies using regex

#### 3 - Inspect the dataset to search for all mentions of movies that won one or more Oscar award (under the `text` subfield in `awards`), but are judged poorly by the viewers (Rotten Tomatoes viewer meter < 50). Select the five movies with the highest total number of awards won.

The proper regex expression is `[Ww]on [0-9]* [Oo]scars?`

This takes into account all following alternatives:
* won/Won
* oscar/Oscar/oscars/Oscars
* any number of Oscars (e.g. 1, 21, 19057...) 

The second method to perform a text search in MongoDB is via the full-text search functionality.
In newer versions of MongoDB this functionality is left for the fully-managed cloud database service offered by MongoDB (Atlas).

In this implementation, the full-text search is implemented using the Apache Lucene project, a Java library (with a Python wrapper) for text indexing and text search, also used in other NoSQL DBs such as Elasticsearch.

In standalone deployments of MongoDB, as the one we are working with, the full-text search functionality can be obtained with a "workaround":

1. Create an index of the field we want to search
2. Perform a query with the `$text` find operator

The `index` is an additional information we can assing to our Collection to ease MongoDB searches.
It can be though as the equivalent of the index of a book. We can use the index to quickly locate the pages where a given appears.
Similarly, in MongoDB we can create an index to quickly identify the documents containing analogous fields.


We first instruct MongoDB to create an index of type `text` associated to a given field:

```python
<DB OBJECT>.<COLLECTION NAME>.create_index([('THE FIELD', 'text')])
```

Then, we can perform a query based on full-text:

```python
<DB OBJECT>.<COLLECTION NAME>.find({ '$text' : {'$search' : "TEXT SEARCH" } })
```


To search for **exact** terms, **escape** the `"` symbols in the search, by using `\"`:

`"\"TEXT SEARCH\""`

To **exclude** terms from a search use the `-` symbols in front of the word to exclude, by using:

`"TEXT SEARCH -WORD"`

The text-search functionality will also provide a `score` for the text search, similarly to any search engine.
We can use this information to sort our text searches by the score. 

Test implementing a full-text search on the `title` field, and search for titles containing the text "Jurassic Park"

In [ ]:
db_mflix.movies.create_index([('title', 'text')])

In [ ]:
# query 
q_ = {'$text' : {'$search': 'Jurassic Park'},
}

# projection
p_ = {'_id':0,
      'title':1,
     }

try:
    _ = db_mflix.movies.find(q_, p_)
    for x in _:
        pprint(x)
except:
    print('Failed query')

Explicitely require the entire phrase to be searched `"\"Jurassic Park\""`

Search for all titles containing "Park" but exclude the word "Jurassic"

Including the score in the sorting of the results is a bit more complex, as it requires adding an additional `score` field defined as a `$meta` variable...

```python
[('score', { "$meta":"textScore" })]
```

It goes definitely beyond the scope of this exercise to unpack and undestand all that... The MongoDB documentation does an excellet work in describing the details for those interested:
https://docs.mongodb.com/manual/core/link-text-indexes/

However, it is interesting to see how we can perfor a Google-like search on arbitrarily complex documents with MongoDB.

## 08 - Aggregation Queries

More complex and articulated queries can be performed via the `$expr` aggregator expression and the Aggregation Pipeline.

The aggregator expression queries, or `$expr`-based, are queries where the conditions are set on the **content** of the fields themselves.

For instance, we can ask to retrieve all documents where the IMDb Rating is higher than the Rotten Tomatoes Critics Rating (e.g. "IMDb Rating > Rotten Tomatoes Critics Rating").

This implies accessing and comparing the values of both fields to perform the query, instead of comparing the value of one field with a "static" and external value (e.g. "IMDb Rating > 8.5")

To perform a query based on the `$expr` condition, we use the syntax:

```python
<DB_OBJECT>.<COLLECTION_NAME>.find( '$expr' : { '<COMPARISON OPERATOR>': [ '$FIELD_1', 
                                                                           '$FIELD_2']}
                                  )
```

* Find 5 movies where the IMDB Rating is higher than the Rotten Tomatoes Critics Rating
    * Make sure the two fields exist in the document (check the `$exists` operator)

In [ ]:
# query 
q_ = {'imdb.rating' : {'$exists': True},
      'tomatoes.critic.rating' : {'$exists': True},
      '$expr' : {'$gt': ['$imdb.rating', '$tomatoes.critic.rating']} }

# projection
p_ = {'_id':0,
      'title':1,
      'imdb':1,
      'tomatoes':1,
     }

# limit
l_ = 5

try:
    _ = db_mflix.movies.find(q_,p_).limit(l_)
    for x in _:
        pprint(x)
except:
    print('Failed query')

To proceed towards further aggregations based on the very powerful and expressive Aggregation Pipeline it is required to dig deeper into the documentation to explore the vast number of aggregator operators: https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/

We will limit to 2 here:
1. match
2. group

`$match` provides a way to filter the data as in a "standard" `find` query operation.

`$group` will perform similar aggregation to the SQL equivalent `GROUP BY`, by combining all records based on the value of a specific field, and evaluating cumulative information (such as sum, count, max, ...)

To work with aggregations in pymongo, we use `aggregate` instead of `find`:

```python
<DB_OBJECT>.<COLLECTION_NAME>.aggregate( [...] )
```

And we pass to the `aggregate` method the list of operations we want to affect our documents.
*The ordering of the aggregations is extremely important*. All aggregation operations will be performed in series, each one acting on the output of the previous.

```python
[{<AGGREGATION OPERATION 1>},
 {<AGGREGATION OPERATION 2>}]
```

The `$match` aggregation operator acts almost identically to a normal `find` query:

```python
{'$match' : {<FIELD 1> : <CONDITION 1>,
             <FIELD 2> : <CONDITION 2>} }
```

To express a groupby in pymongo, we first express the new `"_id"`, the unique field on which to perform the aggregation.
Then, we declare other fields and we match them to the result of aggregation functions such as `$sum` or `$max`.

```python
{'$group' : {'_id' : <$VALUE OF THE FIELD TO GROUP BY>,
             '<NEW AGGREGATE FIELD>' : { $<AGGREGATION OPERATOR> : $<VALUE OF THE FIELD TO AGGREGATE> } 
             } }
```

Let's try to implement a simple query with the MongoDB Aggregation Pipeline.

Let's evaluate, in a single query:

* the distribution of the number of movies versus the Rating set by Rotten Tomatoes Viewers
* _AND_ the average number of Rotten Tomatoes Viewers Reviews versus their Rating
    * filtering only those movies produced after year 2000
        
To work this query out, proceed by performing the following steps:

1. Define the `$match` operator, acting on the `year` feature
2. Define the `$group` operator, with the unique identifier being the *value* (`$`) of `tomatoes.viewer.rating`
3. Include the two aggregations:
    - A `count`  field using the `$sum` operator
    - A `avgRev` field using the `$avg` operator on the *value* (`$`) of `tomatoes.viewer.numReviews`
4. Return the result of the aggregation in a `pandas` DataFrame and plot both distributions as scatter plots

In [ ]:
_ = db_mflix.movies.aggregate([ 
                              # select all movies after 2000
                              {
                                  
                              },
                              # group by the value of tomatoes.viewer.rating
                              {'$group': {
                                      '_id' :   ,
                                      # count all documents
                                      'count' : {   },
                                      # get the average number of reviews per movie
                                      'avgRev': {   },
                                  } 
                              }
                              ])

In [ ]:
df = pd.DataFrame(list(_))

In [ ]:
plt.scatter(x='_id',
            y='count',
            data=df, 
            label='movies after 2000')
plt.xlabel("Rotten Tomatoes Viewers Rating")
plt.ylabel("count")
plt.legend(loc='best')

In [ ]:
plt.scatter(x='_id',
            y='avgRev',
            data=df, 
            label='movies after 2000')
plt.xlabel("Rotten Tomatoes Viewers Rating")
plt.ylabel("Average number of reviews")

1. Plot the average, min and max movie runtime by year. Avoid including documents with type `series`.


2. Find the best rating Metacritic assigned each year, starting from 1995 onwards. Limit the result to only those movies with at least 1 award nomination and more than 1000 IMDb votes.


3. Group movies by directors (use the full list to select also "join ventures" of multiple directors), and draw a scatter plot of the number of their movies vs their average IMDB rating, limiting to directors with more than 10 and less than 50 movies.


4. Select movies that won at least 1 Golden Globe award starting from 1980 (try using `regex`). Group them by the country and plot the number of Golden Globes won per each country.
   * The `countries` field is a list... to properly count all times each country enters in the list we need to use the `$unwind` operator. Check it out on the documentation [here](https://docs.mongodb.com/manual/reference/operator/aggregation/unwind/)

#### 1 - Plot the average, min and max movie runtime by year. Avoid including documents with type `series`.

#### 2 - Find the best rating Metacritic assigned each year, starting from 1995 onwards. Limit the result to only those movies with at least 1 award nomination and more than 1000 IMDb votes.

#### 3 - Group movies by directors (use the full list to select also "join ventures" of multiple directors), and draw a scatter plot of the number of their movies vs their average IMDB rating, limiting to directors with more than 10 and less than 50 movies.

#### 4 - Select movies that won at least 1 Golden Globe award starting from 1980 (try using `regex`). Group them by the country and plot the number of Golden Globes won per each country.